In [ ]:
from cnn_source import Simple3DCNN
import torch
import torch.nn as nn
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('mps') if torch.mps.is_available() else torch.device('cpu')
squat_model_file_1 = 'model/squat_train.pt'
model_squat_1 = Simple3DCNN().to(device)
model_squat_1.load_state_dict(torch.load(squat_model_file_1,map_location=device))
model_file = 'model/classifying_cnn.pt'
model = Simple3DCNN().to(device)

In [ ]:
import cv2
import numpy as np
import torch

# 실시간 데이터 준비 함수
def prepare_realtime_data(cap, frames_per_video=15, height=640, width=480, device='cpu'):
    frames = []

    while len(frames) < frames_per_video:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # BGR -> RGB 변환
        green_frame = frame[:, :, 1]  # 초록색 채널 추출 (H, W)
        green_frame = cv2.resize(green_frame, (width, height))  # 크기 조정
        frames.append(green_frame)

    # 부족한 프레임을 0으로 패딩
    while len(frames) < frames_per_video:
        frames.append(np.zeros((height, width), dtype=np.float32))

    # numpy 배열 -> torch 텐서 변환 및 차원 조정
    frames = torch.tensor(np.array(frames), dtype=torch.float32, device=device)  # (T, H, W)
    frames = frames.unsqueeze(1)  # (T, H, W) -> (T, C, H, W)
    frames = frames.permute(1, 0, 2, 3).unsqueeze(0)  # (T, C, H, W) -> (1, C, T, H, W)

    return frames


In [1]:
import cv2
import time

# 웹캠 캡처 객체 생성
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    exit()


# 카운트다운 시간 설정
countdown_numbers = ["", "", 3, 2, 1, "Start"]

for count in countdown_numbers:
    # 1초 동안 텍스트를 표시
    start_time = time.time()
    while time.time() - start_time <= 1:
        ret, frame = cap.read()

        if not ret:
            break

        # 화면 크기
        height, width, _ = frame.shape

        # 텍스트 설정
        text = str(count)
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 2
        font_thickness = 5
        color = (255, 255, 255)

        # 텍스트 중앙 배치
        text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
        text_x = (width - text_size[0]) // 2
        text_y = (height + text_size[1]) // 2

        # 텍스트 추가
        cv2.putText(frame, text, (text_x, text_y), font, font_scale, color, font_thickness)

        # 화면 표시
        with torch.no_grad():
            while True:
            # 실시간 데이터 준비
                inputs = prepare_realtime_data(cap, frames_per_video=15, device=device)

            # 모델에 입력
                outputs = model(inputs)

        # 출력 처리
                predicted_class = torch.argmax(outputs, dim=1)
                False
        if predicted_class == 0 :
            pose = 'Push'
        elif predicted_class == 1 :                    
            pose = 'squat'
        elif predicted_class == 2 :
            pose = 'lunge'
        else :
            pose = 'pose not included'     
        # 'q'를 누르면 종료
        if cv2.waitKey(1) == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            exit()

# 텍스트 사라진 화면 유지
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 화면 표시 (텍스트 없음)
    cv2.imshow("Camera", frame)

    # 'q'를 누르면 종료
    if cv2.waitKey(1) == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()